# 导入工具包

In [10]:
# opencv-python
import cv2

# mediapipe人工智能工具包
import mediapipe as mp

# 进度条库
from tqdm import tqdm

# 时间库
import time

# 导入模型

In [11]:
# 导入人脸检测模型
mp_face_detection = mp.solutions.face_detection
model = mp_face_detection.FaceDetection(   
        min_detection_confidence=0.5, # 置信度阈值，过滤掉小于置信度的预测框
        model_selection=0,            # 选择模型，0适用于人脸距离镜头近（2米以内），1适用于人脸距离镜头远（5米以内）
)

In [12]:
# 导入可视化函数和可视化样式
mp_drawing = mp.solutions.drawing_utils
# 关键点样式
keypoint_style = mp_drawing.DrawingSpec(thickness=5, circle_radius=3, color=(0,255,0))
# 人脸预测框样式
bbox_style = mp_drawing.DrawingSpec(thickness=5, circle_radius=3, color=(255,0,0))

# 处理单帧的函数

In [16]:
# 处理帧函数
def process_frame(img):
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    # BGR转RGB
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 将RGB图像输入模型，获取预测结果
    results = model.process(img_RGB)
    
    if results.detections: # 当检测出人脸的时候
        # 可视化人脸框和人脸关键点
        for detection in results.detections:
            mp_drawing.draw_detection(img, 
                                      detection,
                                      keypoint_drawing_spec=keypoint_style,
                                      bbox_drawing_spec=bbox_style)
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    
    scaler = 1
    # 在图像上写FPS数值，参数依次为：图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
    img = cv2.putText(img, 'FPS  '+str(int(FPS)), (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)

    return img

# 调用摄像头获取每帧（模板）

In [17]:
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需修改process_frame函数即可

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture(1)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    # 获取画面
    success, frame = cap.read()
    if not success:
        print('Error')
        break
    
    ## !!!处理帧函数
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)

    if cv2.waitKey(1) in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()